In [ ]:
!pip install wandb
!pip install category_encoders

     |████████████████████████████████| 1.4MB 3.5MB/s 
     |████████████████████████████████| 102kB 11.3MB/s 
     |████████████████████████████████| 102kB 11.4MB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
     |████████████████████████████████| 460kB 44.4MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 71kB 9.1MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=f54be8ab2837874f2b964ff43da1ba980abc28ca80f660cf7bb2f0c43df9e8a9
  Stored in directory: /root/.cache/pip/wheels/bc/ed/6c/028dea90d31b359cd2a7c8b0da4db80e41d24a59614154072e
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=45d69263c92f58c924851a097e7148e165b2f4a4cb91a2518e7c97a402f69f1f
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=73

In [ ]:
"""
Import Statements:
"""

# Classics 
import numpy as np
import pandas as pd


# Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import wandb
from wandb.keras import WandbCallback
from category_encoders import TargetEncoder
from category_encoders import OrdinalEncoder
from tensorflow import keras

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving medcab1_2.csv to medcab1_2 (2).csv


In [ ]:
df = pd.read_csv('medcab1_2.csv')
df = df.fillna('none')
df = df.replace([True, False],
                    [1.0, 0.0])
df = df.drop('Unnamed: 0', 1)
print(df.shape)
df.head(3)

(2350, 73)


,index,Strain,Type,Rating,Effects,Flavor,Description,Ammonia_flavor,Apple_flavor,Apricot_flavor,Berry_flavor,Blue_flavor,Blueberry_flavor,Citrus_flavor,Cheese_flavor,Chemical_flavor,Chestnut_flavor,Diesel_flavor,Earthy_flavor,Flowery_flavor,Fruit_flavor,Grape_flavor,Grapefruit_flavor,Honey_flavor,Lavender_flavor,Lemon_flavor,Mango_flavor,Menthol_flavor,Mint_flavor,Minty_flavor,Nutty_flavor,Orange_flavor,Peach_flavor,Pepper_flavor,Pine_flavor,Pineapple_flavor,Pungent_flavor,Sage_flavor,Skunk_flavor,SpicyHerbal_flavor,Strawberry,Sweet_flavor,Tea_flavor,Tobacco_flavor,Tree_flavor,Tropical_flavor,Vanilla_flavor,Violet_flavor,Woody_flavor,Aroused_effect,Creative_effect,Energetic_effect,Euphoric_effect,Focused_effect,Giggly_effect,Happy_effect,Hungry_effect,Relaxed_effect,Sleepy_effect,Talkative_effect,Tingly_effect,Uplifted_effect,None_effect,hybrid_type,sativa_type,indica_type,Savor,Style,Breed,Text,Effects_and_Flavor,ID,tokens
0,0.0,100-Og,hybrid,4.0,"Creative, Energetic, Tingly, Euphoric, Relaxed","Earthy, Sweet, Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,", Earthy, Sweet, Citrus",", $100 OG is a 50/50 hybrid strain that packs ...",", 100-Og",Creative Energetic Tingly Euphoric Relaxed...,"Creative, Energetic, Tingly, Euphoric, Relaxed...",0.0,"['creative', 'energetic', 'tingly', 'euphoric'..."
1,1.0,98-White-Widow,hybrid,4.7,"Relaxed, Aroused, Creative, Happy, Energetic","Flowery, Violet, Diesel",The ‘98 Aloha White Widow is an especially pot...,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,", Flowery, Violet, Diesel",", The ‘98 Aloha White Widow is an especially p...",", 98-White-Widow",Relaxed Aroused Creative Happy Energetic ...,"Relaxed, Aroused, Creative, Happy, Energetic, ...",1.0,"['relaxed', 'aroused', 'creative', 'happy', 'e..."
2,2.0,1024,sativa,4.4,"Uplifted, Happy, Relaxed, Energetic, Creative","Spicy/Herbal, Sage, Woody",1024 is a sativa-dominant hybrid bred in Spain...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,", Spicy/Herbal, Sage, Woody",", 1024 is a sativa-dominant hybrid bred in Spa...",", 1024",Uplifted Happy Relaxed Energetic Creative ...,"Uplifted, Happy, Relaxed, Energetic, Creative,...",2.0,"['uplifted', 'happy', 'relaxed', 'energetic', ..."


In [ ]:
df.isnull().sum()

index                 0
Strain                0
Type                  0
Rating                0
Effects               0
                     ..
Breed                 0
Text                  0
Effects_and_Flavor    0
ID                    0
tokens                0
Length: 73, dtype: int64

In [ ]:
df_rating = df['Rating']/100

In [ ]:
df_rating

0       0.040
1       0.047
2       0.044
3       0.042
4       0.046
        ...  
2345    0.047
2346    0.046
2347    0.050
2348    0.044
2349    0.046
Name: Rating, Length: 2350, dtype: float64

In [ ]:
df['Strain'] = df.index

In [ ]:
#df.to_pickle
for col in df.columns:
  print(col)

index
Strain
Type
Rating
Effects
Flavor
Description
Ammonia_flavor
Apple_flavor
Apricot_flavor
Berry_flavor
Blue_flavor
Blueberry_flavor
Citrus_flavor
Cheese_flavor
Chemical_flavor
Chestnut_flavor
Diesel_flavor
Earthy_flavor
Flowery_flavor
Fruit_flavor
Grape_flavor
Grapefruit_flavor
Honey_flavor
Lavender_flavor
Lemon_flavor
Mango_flavor
Menthol_flavor
Mint_flavor
Minty_flavor
Nutty_flavor
Orange_flavor
Peach_flavor
Pepper_flavor
Pine_flavor
Pineapple_flavor
Pungent_flavor
Sage_flavor
Skunk_flavor
SpicyHerbal_flavor
Strawberry
Sweet_flavor
Tea_flavor
Tobacco_flavor
Tree_flavor
Tropical_flavor
Vanilla_flavor
Violet_flavor
Woody_flavor
Aroused_effect
Creative_effect
Energetic_effect
Euphoric_effect
Focused_effect
Giggly_effect
Happy_effect
Hungry_effect
Relaxed_effect
Sleepy_effect
Talkative_effect
Tingly_effect
Uplifted_effect
None_effect
hybrid_type
sativa_type
indica_type
Savor
Style
Breed
Text
Effects_and_Flavor
ID
tokens


In [ ]:
df.columns

Index(['index', 'Strain', 'Type', 'Rating', 'Effects', 'Flavor', 'Description',
       'Ammonia_flavor', 'Apple_flavor', 'Apricot_flavor', 'Berry_flavor',
       'Blue_flavor', 'Blueberry_flavor', 'Citrus_flavor', 'Cheese_flavor',
       'Chemical_flavor', 'Chestnut_flavor', 'Diesel_flavor', 'Earthy_flavor',
       'Flowery_flavor', 'Fruit_flavor', 'Grape_flavor', 'Grapefruit_flavor',
       'Honey_flavor', 'Lavender_flavor', 'Lemon_flavor', 'Mango_flavor',
       'Menthol_flavor', 'Mint_flavor', 'Minty_flavor', 'Nutty_flavor',
       'Orange_flavor', 'Peach_flavor', 'Pepper_flavor', 'Pine_flavor',
       'Pineapple_flavor', 'Pungent_flavor', 'Sage_flavor', 'Skunk_flavor',
       'SpicyHerbal_flavor', 'Strawberry', 'Sweet_flavor', 'Tea_flavor',
       'Tobacco_flavor', 'Tree_flavor', 'Tropical_flavor', 'Vanilla_flavor',
       'Violet_flavor', 'Woody_flavor', 'Aroused_effect', 'Creative_effect',
       'Energetic_effect', 'Euphoric_effect', 'Focused_effect',
       'Giggly_effect', 'Ha

In [ ]:
target = 'Strain'
features = df[['Type', 'Rating', 'Effects', 'Flavor', 'Description',
       'Ammonia_flavor', 'Apple_flavor', 'Apricot_flavor', 'Berry_flavor',
       'Blue_flavor', 'Blueberry_flavor', 'Citrus_flavor', 'Cheese_flavor',
       'Chemical_flavor', 'Chestnut_flavor', 'Diesel_flavor', 'Earthy_flavor',
       'Flowery_flavor', 'Fruit_flavor', 'Grape_flavor', 'Grapefruit_flavor',
       'Honey_flavor', 'Lavender_flavor', 'Lemon_flavor', 'Mango_flavor',
       'Menthol_flavor', 'Mint_flavor', 'Minty_flavor', 'Nutty_flavor',
       'Orange_flavor', 'Peach_flavor', 'Pepper_flavor', 'Pine_flavor',
       'Pineapple_flavor', 'Pungent_flavor', 'Sage_flavor', 'Skunk_flavor',
       'SpicyHerbal_flavor', 'Strawberry', 'Sweet_flavor', 'Tea_flavor',
       'Tobacco_flavor', 'Tree_flavor', 'Tropical_flavor', 'Vanilla_flavor',
       'Violet_flavor', 'Woody_flavor', 'Aroused_effect', 'Creative_effect',
       'Energetic_effect', 'Euphoric_effect', 'Focused_effect',
       'Giggly_effect', 'Happy_effect', 'Hungry_effect', 'Relaxed_effect',
       'Sleepy_effect', 'Talkative_effect', 'Tingly_effect', 'Uplifted_effect',
       'None_effect', 'hybrid_type', 'sativa_type', 'indica_type', 'Savor',
       'Style', 'Breed', 'Text', 'Effects_and_Flavor',]]

train, test = train_test_split(df, train_size=0.75, test_size=0.25, random_state=42)

In [ ]:
print(train.shape)
train

(1762, 73)


,index,Strain,Type,Rating,Effects,Flavor,Description,Ammonia_flavor,Apple_flavor,Apricot_flavor,Berry_flavor,Blue_flavor,Blueberry_flavor,Citrus_flavor,Cheese_flavor,Chemical_flavor,Chestnut_flavor,Diesel_flavor,Earthy_flavor,Flowery_flavor,Fruit_flavor,Grape_flavor,Grapefruit_flavor,Honey_flavor,Lavender_flavor,Lemon_flavor,Mango_flavor,Menthol_flavor,Mint_flavor,Minty_flavor,Nutty_flavor,Orange_flavor,Peach_flavor,Pepper_flavor,Pine_flavor,Pineapple_flavor,Pungent_flavor,Sage_flavor,Skunk_flavor,SpicyHerbal_flavor,Strawberry,Sweet_flavor,Tea_flavor,Tobacco_flavor,Tree_flavor,Tropical_flavor,Vanilla_flavor,Violet_flavor,Woody_flavor,Aroused_effect,Creative_effect,Energetic_effect,Euphoric_effect,Focused_effect,Giggly_effect,Happy_effect,Hungry_effect,Relaxed_effect,Sleepy_effect,Talkative_effect,Tingly_effect,Uplifted_effect,None_effect,hybrid_type,sativa_type,indica_type,Savor,Style,Breed,Text,Effects_and_Flavor,ID,tokens
1545,1545.0,1545,indica,4.8,"Relaxed, Giggly, Uplifted, Euphoric, Sleepy","Spicy/Herbal, Pepper, Earthy",Ortega by Mr. Nice Seeds is an indica strain b...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,", Spicy/Herbal, Pepper, Earthy",", Ortega by Mr. Nice Seeds is an indica strain...",", Ortega",Relaxed Giggly Uplifted Euphoric Sleepy S...,"Relaxed, Giggly, Uplifted, Euphoric, Sleepy, S...",1545.0,"['relaxed', 'giggly', 'uplifted', 'euphoric', ..."
1347,1347.0,1347,hybrid,5.0,"Talkative, Energetic, Giggly, Relaxed","Sweet, Berry, Earthy",Maramota #10 is a concoction of DJ Short Blueb...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,", Sweet, Berry, Earthy",", Maramota #10 is a concoction of DJ Short Blu...",", Maramota-10",Talkative Energetic Giggly Relaxed Sweet ...,"Talkative, Energetic, Giggly, Relaxed, Sweet, ...",1347.0,"['talkative', 'energetic', 'giggly', 'relaxed'..."
543,543.0,543,hybrid,4.6,"Energetic, Relaxed, Euphoric, Uplifted, Happy","Diesel, Pungent, Earthy",Chunky Diesel bred Ganja Rebel Seed Co. for Cr...,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,", Diesel, Pungent, Earthy",", Chunky Diesel bred Ganja Rebel Seed Co. for ...",", Chunky-Diesel",Energetic Relaxed Euphoric Uplifted Happy ...,"Energetic, Relaxed, Euphoric, Uplifted, Happy,...",543.0,"['energetic', 'relaxed', 'euphoric', 'uplifted..."
1533,1533.0,1533,indica,4.1,"Relaxed, Happy, Euphoric, Sleepy, Hungry","Orange, Citrus, Sweet",Orange Kush was bred by Green Devil Genetics a...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,", Orange, Citrus, Sweet",", Orange Kush was bred by Green Devil Genetics...",", Orange-Kush",Relaxed Happy Euphoric Sleepy Hungry Oran...,"Relaxed, Happy, Euphoric, Sleepy, Hungry, Oran...",1533.0,"['relaxed', 'happy', 'euphoric', 'sleepy', 'hu..."
494,494.0,494,hybrid,4.8,"Uplifted, Euphoric, Relaxed, Happy, Energetic","Diesel, Sweet, Citrus",Chemnesia has been dubbed one of the most powe...,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,", Diesel, Sweet, Citrus",", Chemnesia has been dubbed one of the most po...",", Chemnesia",Uplifted Euphoric Relaxed Happy Energetic ...,"Uplifted, Euphoric, Relaxed, Happy, Energetic,...",494.0,"['uplifted', 'euphoric', 'relaxed', 'happy', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,1638.0,1638

In [ ]:
#Strain input and output
target = df['Strain']
feautures = df[['Type', 'Rating', 'Effects', 'Flavor', 'Description',
       'Ammonia_flavor', 'Apple_flavor', 'Apricot_flavor', 'Berry_flavor',
       'Blue_flavor', 'Blueberry_flavor', 'Citrus_flavor', 'Cheese_flavor',
       'Chemical_flavor', 'Chestnut_flavor', 'Diesel_flavor', 'Earthy_flavor',
       'Flowery_flavor', 'Fruit_flavor', 'Grape_flavor', 'Grapefruit_flavor',
       'Honey_flavor', 'Lavender_flavor', 'Lemon_flavor', 'Mango_flavor',
       'Menthol_flavor', 'Mint_flavor', 'Minty_flavor', 'Nutty_flavor',
       'Orange_flavor', 'Peach_flavor', 'Pepper_flavor', 'Pine_flavor',
       'Pineapple_flavor', 'Pungent_flavor', 'Sage_flavor', 'Skunk_flavor',
       'SpicyHerbal_flavor', 'Strawberry', 'Sweet_flavor', 'Tea_flavor',
       'Tobacco_flavor', 'Tree_flavor', 'Tropical_flavor', 'Vanilla_flavor',
       'Violet_flavor', 'Woody_flavor', 'Aroused_effect', 'Creative_effect',
       'Energetic_effect', 'Euphoric_effect', 'Focused_effect',
       'Giggly_effect', 'Happy_effect', 'Hungry_effect', 'Relaxed_effect',
       'Sleepy_effect', 'Talkative_effect', 'Tingly_effect', 'Uplifted_effect',
       'None_effect', 'hybrid_type', 'sativa_type', 'indica_type', 'Savor',
       'Style', 'Breed', 'Text', 'Effects_and_Flavor',]]
#Ratings should be standerized

In [ ]:
train.describe(exclude='number')

,Type,Effects,Flavor,Description,Ammonia_flavor,Apple_flavor,Apricot_flavor,Berry_flavor,Blue_flavor,Blueberry_flavor,Citrus_flavor,Cheese_flavor,Chemical_flavor,Chestnut_flavor,Diesel_flavor,Earthy_flavor,Flowery_flavor,Fruit_flavor,Grape_flavor,Grapefruit_flavor,Honey_flavor,Lavender_flavor,Lemon_flavor,Mango_flavor,Menthol_flavor,Mint_flavor,Minty_flavor,Nutty_flavor,Orange_flavor,Peach_flavor,Pepper_flavor,Pine_flavor,Pineapple_flavor,Pungent_flavor,Sage_flavor,Skunk_flavor,SpicyHerbal_flavor,Strawberry,Sweet_flavor,Tea_flavor,Tobacco_flavor,Tree_flavor,Tropical_flavor,Vanilla_flavor,Violet_flavor,Woody_flavor,Savor,Style,Breed,Text,Effects_and_Flavor,tokens
count,1762,1762,1762,1762,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762,1762,1762,1762,1762,1762
unique,3,1318,1042,1737,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,1042,1737,1762,1762,1700,1762
top,hybrid,None,None,none,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,", None",", nan",", White-Smurf",Happy Euphoric Relaxed Giggly Tingly Citr...,"None, None","['euphoric', 'happy', 'uplifted', 'energetic',..."
freq,915,62,76,25,1702.0,1713.0,1718.0,1463.0,1608.0,1614.0,1328.0,1675.0,1693.0,1718.0,1538.0,903.0,1518.0,1701.0,1601.0,1696.0,1696.0,1696.0,1573.0,1701.0,1707.0,1681.0,1693.0,1706.0,1666.0,1719.0,1680.0,1469.0,1693.0,1399.0,1697.0,1597.0,1563.0,1687.0,939.0,1711.0,1719.0,1701.0,1611.0,1697.0,1717.0,1528.0,76,25,1,1,52,1


In [ ]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
train.dtypes

index                 float64
Strain                  int64
Type                   object
Rating                float64
Effects                object
                       ...   
Breed                  object
Text                   object
Effects_and_Flavor     object
ID                    float64
tokens                 object
Length: 73, dtype: object

In [ ]:
train['Text'][0]

'Creative  Energetic  Tingly  Euphoric  Relaxed  Earthy  Sweet  Citrus  $100 OG is a 50/50 hybrid strain that packs a strong punch. The name supposedly refers to both its strength and high price when it first started showing up in Hollywood. As a plant  $100 OG tends to produce large dark green buds with few stems. Users report a strong body effect of an indica for pain relief with the more alert  cerebral feeling thanks to its sativa side.  100-Og  $100 OG is a 50/50 hybrid strain that packs a strong punch. The name supposedly refers to both its strength and high price when it first started showing up in Hollywood. As a plant  $100 OG tends to produce large dark green buds with few stems. Users report a strong body effect of an indica for pain relief with the more alert  cerebral feeling thanks to its sativa side.'

In [ ]:
#Dataset
from sklearn.datasets import fetch_20newsgroups

categories =['alt.atheism',
            'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [ ]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [ ]:
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect),
                 # Classifier
                 ('clf', rfc)
                ])

# The pipeline puts together a bunch fit then transform,fit then predict. 

In [ ]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   26.7s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.5min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [ ]:
def fit(self, X, *args, **kwargs):
        if self._grid_search:
            model = GridSearchCV(self._model, **self._grid_search)
        elif self._random_search:
            model = RandomizedSearchCV(self._model, **self._random_search)
        else:
            model = self._model

        if self._grid_search is not None:
            self._grid = model
        elif self._random_search is not None:
            self._rnd = model

        assert (self.target in X.columns.values), 'X must contain the target column'
        self._xcols = list(X.columns.values)
        self._xcols.remove(self.target)
        if len(self._columns_exclude) == 0 and len(self._columns_include) > 0:
            self._columns_exclude = list(set(self._xcols) - set(self._columns_include))
        [self._xcols.remove(t) for t in self._columns_exclude]
        x = X[self._xcols]
        y = X[self.target]
        model.fit(x, y, **kwargs)
        return self 

In [ ]:
grid_search.best_score_

0.8821501427988576

In [ ]:
grid_search.predict(['Send me lots of money now', 'you won the lottery in Nigeria'])

array([1, 1])

In [ ]:
train['Text'][0]

'Creative  Energetic  Tingly  Euphoric  Relaxed  Earthy  Sweet  Citrus  $100 OG is a 50/50 hybrid strain that packs a strong punch. The name supposedly refers to both its strength and high price when it first started showing up in Hollywood. As a plant  $100 OG tends to produce large dark green buds with few stems. Users report a strong body effect of an indica for pain relief with the more alert  cerebral feeling thanks to its sativa side.  100-Og  $100 OG is a 50/50 hybrid strain that packs a strong punch. The name supposedly refers to both its strength and high price when it first started showing up in Hollywood. As a plant  $100 OG tends to produce large dark green buds with few stems. Users report a strong body effect of an indica for pain relief with the more alert  cerebral feeling thanks to its sativa side.'

### Define Pipeline Components

In [ ]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

In [ ]:
help(TfidfVectorizer)

Help on class TfidfVectorizer in module sklearn.feature_extraction.text:

class TfidfVectorizer(CountVectorizer)
 |  Convert a collection of raw documents to a matrix of TF-IDF features.
 |  
 |  Equivalent to :class:`CountVectorizer` followed by
 |  :class:`TfidfTransformer`.
 |  
 |  Read more in the :ref:`User Guide <text_feature_extraction>`.
 |  
 |  Parameters
 |  ----------
 |  input : str {'filename', 'file', 'content'}
 |      If 'filename', the sequence passed as an argument to fit is
 |      expected to be a list of filenames that need reading to fetch
 |      the raw content to analyze.
 |  
 |      If 'file', the sequence items must have a 'read' method (file-like
 |      object) that is called to fetch the bytes in memory.
 |  
 |      Otherwise the input is expected to be a sequence of items that
 |      can be of type string or byte.
 |  
 |  encoding : str, default='utf-8'
 |      If bytes or files are given to analyze, this encoding is used to
 |      decode.
 |  
 | 

In [ ]:
tokenize = vect.build_tokenizer()
stop_words = vect.get_stop_words()

In [ ]:
grid_search.best_params_

{'clf__max_depth': 20,
 'clf__n_estimators': 10,
 'vect__max_df': 0.75,
 'vect__max_features': 1000,
 'vect__min_df': 0.02}

In [ ]:
grid_search.best_score_

0.8821501427988576

### Define our search space


In [ ]:
train.describe(exclude='number')

,Type,Effects,Flavor,Description,Ammonia_flavor,Apple_flavor,Apricot_flavor,Berry_flavor,Blue_flavor,Blueberry_flavor,Citrus_flavor,Cheese_flavor,Chemical_flavor,Chestnut_flavor,Diesel_flavor,Earthy_flavor,Flowery_flavor,Fruit_flavor,Grape_flavor,Grapefruit_flavor,Honey_flavor,Lavender_flavor,Lemon_flavor,Mango_flavor,Menthol_flavor,Mint_flavor,Minty_flavor,Nutty_flavor,Orange_flavor,Peach_flavor,Pepper_flavor,Pine_flavor,Pineapple_flavor,Pungent_flavor,Sage_flavor,Skunk_flavor,SpicyHerbal_flavor,Strawberry,Sweet_flavor,Tea_flavor,Tobacco_flavor,Tree_flavor,Tropical_flavor,Vanilla_flavor,Violet_flavor,Woody_flavor,Savor,Style,Breed,Text,Effects_and_Flavor,tokens
count,1762,1762,1762,1762,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762.0,1762,1762,1762,1762,1762,1762
unique,3,1318,1042,1737,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,1042,1737,1762,1762,1700,1762
top,hybrid,None,None,none,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,", None",", nan",", White-Smurf",Happy Euphoric Relaxed Giggly Tingly Citr...,"None, None","['euphoric', 'happy', 'uplifted', 'energetic',..."
freq,915,62,76,25,1702.0,1713.0,1718.0,1463.0,1608.0,1614.0,1328.0,1675.0,1693.0,1718.0,1538.0,903.0,1518.0,1701.0,1601.0,1696.0,1696.0,1696.0,1573.0,1701.0,1707.0,1681.0,1693.0,1706.0,1666.0,1719.0,1680.0,1469.0,1693.0,1399.0,1697.0,1597.0,1563.0,1687.0,939.0,1711.0,1719.0,1701.0,1611.0,1697.0,1717.0,1528.0,76,25,1,1,52,1


### Decision Tree Classifier on Effects:

In [ ]:
# family of variables

train = df
test = df
val = df

target = ['Strain']
features = ['Effects']
#features = ['Effects', 'Flavors']

In [ ]:
test2_features = ['Text']

In [ ]:
type(test2_features)

list

In [ ]:
test3_features = []
test3_f = df.loc[1, 'Text'] 
test3_features.append(test3_f)

In [ ]:
type(test3_features)

list

In [ ]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [ ]:
X_test2 = test[test2_features]

In [ ]:
X_test3 = test3_features

In [ ]:
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.tree import DecisionTreeClassifier
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    #SimpleImputer(strategy='mean'), 
    DecisionTreeClassifier(random_state=42)
)

In [ ]:
# Fit on train
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('onehotencoder',
                 OneHotEncoder(cols=['Effects'], drop_invariant=False,
                               handle_missing='value', handle_unknown='value',
                               return_df=True, use_cat_names=True, verbose=0)),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features=None, max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        presort='deprecated', random_state=42,
                                        splitter='best'))],
         verbose=Fals

In [ ]:
# Score on train, val
print('Train Accuracy', pipeline.score(X_train, y_train))

Train Accuracy 0.7042553191489361


In [ ]:
# Predict on test
y_pred = pipeline.predict(X_test)

In [ ]:
X_test.head()

,Effects
0,"Creative, Energetic, Tingly, Euphoric, Relaxed"
1,"Relaxed, Aroused, Creative, Happy, Energetic"
2,"Uplifted, Happy, Relaxed, Energetic, Creative"
3,"Tingly, Creative, Hungry, Relaxed, Uplifted"
4,"Happy, Relaxed, Euphoric, Uplifted, Talkative"


### Spacy

In [ ]:
import spacy
# Import the English language class
from spacy.lang.en import English

# Create the nlp object
nlp = English()

In [ ]:
df.loc[1, 'Description']

'The ‘98 Aloha White Widow is an especially potent cut of White Widow that has grown in renown alongside Hawaiian legends like Maui Wowie and Kona Gold. This White Widow phenotype reeks of diesel and skunk and has a rich earthy taste with intermittent notes of hash. Its buds are coated in trichomes  giving its dark foliage a lustrous glint to go along with its room-filling odor. This one-hitter-quitter uplifts the mind with mind-bending euphoria that materializes in the body as airy relaxation. ‘98 Aloha White Widow is available from Pua Mana 1st Hawaiian Pakalōlō Seed Bank. \xa0'

In [ ]:
text1 = df.loc[1, 'Description']


# Created by processing a string of text with the nlp object
doc = nlp(text1)

# Iterate over tokens in a Doc
for token in doc:
    print(token.text)

The
‘
98
Aloha
White
Widow
is
an
especially
potent
cut
of
White
Widow
that
has
grown
in
renown
alongside
Hawaiian
legends
like
Maui
Wowie
and
Kona
Gold
.
This
White
Widow
phenotype
reeks
of
diesel
and
skunk
and
has
a
rich
earthy
taste
with
intermittent
notes
of
hash
.
Its
buds
are
coated
in
trichomes
 
giving
its
dark
foliage
a
lustrous
glint
to
go
along
with
its
room
-
filling
odor
.
This
one
-
hitter
-
quitter
uplifts
the
mind
with
mind
-
bending
euphoria
that
materializes
in
the
body
as
airy
relaxation
.
‘
98
Aloha
White
Widow
is
available
from
Pua
Mana
1st
Hawaiian
Pakalōlō
Seed
Bank
.
 


In [ ]:
print('Index:   ', [token.i for token in doc])
print('Text:    ', [token.text for token in doc])
print('is_alpha:', [token.is_alpha for token in doc])
print('is_punct:', [token.is_punct for token in doc])
print('like_num:', [token.like_num for token in doc])

Index:    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113]
Text:     ['The', '‘', '98', 'Aloha', 'White', 'Widow', 'is', 'an', 'especially', 'potent', 'cut', 'of', 'White', 'Widow', 'that', 'has', 'grown', 'in', 'renown', 'alongside', 'Hawaiian', 'legends', 'like', 'Maui', 'Wowie', 'and', 'Kona', 'Gold', '.', 'This', 'White', 'Widow', 'phenotype', 'reeks', 'of', 'diesel', 'and', 'skunk', 'and', 'has', 'a', 'rich', 'earthy', 'taste', 'with', 'intermittent', 'notes', 'of', 'hash', '.', 'Its', 'buds', 'are', 'coated', 'in', 'trichomes', ' ', 'giving', 'its', 'dark', 'foliage', 'a', 'l

In [ ]:
# Load the small English model
nlp = spacy.load('en_core_web_sm')

# Process a text
doc = nlp(text1)

# Iterate over the tokens
for token in doc:
    # Print the text and the predicted part-of-speech tag
    print(token.text, token.pos_)

The DET
‘ ADJ
98 NUM
Aloha PROPN
White PROPN
Widow PROPN
is VERB
an DET
especially ADV
potent ADJ
cut NOUN
of ADP
White PROPN
Widow PROPN
that DET
has VERB
grown VERB
in ADP
renown NOUN
alongside ADP
Hawaiian ADJ
legends NOUN
like ADP
Maui PROPN
Wowie PROPN
and CCONJ
Kona PROPN
Gold PROPN
. PUNCT
This DET
White PROPN
Widow PROPN
phenotype VERB
reeks NOUN
of ADP
diesel NOUN
and CCONJ
skunk NOUN
and CCONJ
has VERB
a DET
rich ADJ
earthy ADJ
taste NOUN
with ADP
intermittent ADJ
notes NOUN
of ADP
hash NOUN
. PUNCT
Its DET
buds NOUN
are VERB
coated VERB
in ADP
trichomes NOUN
  SPACE
giving VERB
its DET
dark ADJ
foliage NOUN
a DET
lustrous ADJ
glint NOUN
to PART
go VERB
along PART
with ADP
its DET
room NOUN
- PUNCT
filling VERB
odor NOUN
. PUNCT
This DET
one NUM
- PUNCT
hitter NOUN
- PUNCT
quitter NOUN
uplifts NOUN
the DET
mind NOUN
with ADP
mind NOUN
- PUNCT
bending VERB
euphoria NOUN
that DET
materializes VERB
in ADP
the DET
body NOUN
as ADP
airy NOUN
relaxation NOUN
. PUNCT
‘ PUNCT
98 NUM


In [ ]:
for token in doc:
    print(token.text, token.pos_, token.dep_, token.head.text)

The DET det Widow
‘ ADJ punct Widow
98 NUM nummod Widow
Aloha PROPN compound Widow
White PROPN compound Widow
Widow PROPN nsubj is
is VERB ROOT is
an DET det cut
especially ADV advmod potent
potent ADJ amod cut
cut NOUN attr is
of ADP prep cut
White PROPN compound Widow
Widow PROPN pobj of
that DET nsubj grown
has VERB aux grown
grown VERB relcl cut
in ADP prep grown
renown NOUN pobj in
alongside ADP prep grown
Hawaiian ADJ amod legends
legends NOUN pobj alongside
like ADP prep legends
Maui PROPN compound Wowie
Wowie PROPN pobj like
and CCONJ cc Wowie
Kona PROPN compound Gold
Gold PROPN conj Wowie
. PUNCT punct is
This DET det phenotype
White PROPN compound Widow
Widow PROPN compound phenotype
phenotype VERB compound reeks
reeks NOUN ROOT reeks
of ADP prep reeks
diesel NOUN pobj of
and CCONJ cc diesel
skunk NOUN conj diesel
and CCONJ cc reeks
has VERB conj reeks
a DET det taste
rich ADJ amod taste
earthy ADJ amod taste
taste NOUN dobj has
with ADP prep taste
intermittent ADJ amod notes

In [ ]:
nlp = spacy.load("en_core_web_sm")

# Process the text
doc = nlp(text1)

for token in doc:
    # Get the token text, part-of-speech tag and dependency label
    token_text = token.text
    token_pos = token.pos_
    token_dep = token.dep_
    # This is for formatting only
    print("{:<12}{:<10}{:<10}".format(token_text, token_pos, token_dep))

The         DET       det       
‘           ADJ       punct     
98          NUM       nummod    
Aloha       PROPN     compound  
White       PROPN     compound  
Widow       PROPN     nsubj     
is          VERB      ROOT      
an          DET       det       
especially  ADV       advmod    
potent      ADJ       amod      
cut         NOUN      attr      
of          ADP       prep      
White       PROPN     compound  
Widow       PROPN     pobj      
that        DET       nsubj     
has         VERB      aux       
grown       VERB      relcl     
in          ADP       prep      
renown      NOUN      pobj      
alongside   ADP       prep      
Hawaiian    ADJ       amod      
legends     NOUN      pobj      
like        ADP       prep      
Maui        PROPN     compound  
Wowie       PROPN     pobj      
and         CCONJ     cc        
Kona        PROPN     compound  
Gold        PROPN     conj      
.           PUNCT     punct     
This        DET       det       
White     

In [ ]:
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

doc = nlp(text1)

# Write a pattern for adjective plus one or two nouns
pattern = [{"POS": 'ADJ'}, 
#           {"POS": 'DET'}, 
           {"POS": 'NOUN', "OP":'+'}]

# Add the pattern to the matcher and apply the matcher to the doc
matcher.add("ADJ_NOUN_PATTERN", None, pattern)
matches = matcher(doc)
print("Total matches found:", len(matches))

# Iterate over the matches and print the span text
for match_id, start, end in matches:
    print("Match found:", doc[start:end].text)

Total matches found: 6
Match found: potent cut
Match found: Hawaiian legends
Match found: earthy taste
Match found: intermittent notes
Match found: dark foliage
Match found: lustrous glint
